## Using the DFT Hamiltonian to learn activation barriers for chemical reactions

In [1]:
import numpy as np
import plotly.express as px
from instance_mongodb import instance_mongodb_sei

from pymatgen.core.structure import Molecule

import torch

from e3nn import o3

from utils import rotate_three_dimensions, subdiagonalize_matrix

DFT computes the energies by solving the generalised eigenvalue problem:

$$
\boldsymbol{H(R)}\psi_i = \epsilon_i \boldsymbol{S(R)}\psi_i
$$

where $\boldsymbol{H(R)}$ is the Hamiltonian Matrix, $\boldsymbol{S(R)}$ is the overlap matrix, $\epsilon_i$ is the computed eigenvalue for state i, with an eigenvector $\psi_i$.

$\boldsymbol{H(R)}$ and $\boldsymbol{S(R)}$ change with rotation, which means we have to process them appropriately prior to using them as features in our NN models.

In [2]:
def add_bounds_to_figure(fig):

    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=4.5,
        x1=6.5,
        y1=4.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=5.5,
        x1=6.5,
        y1=5.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=-0.5,
        x1=-0.5,
        y1=6.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=4.5,
        y0=-0.5,
        x1=4.5,
        y1=6.5,
        line=dict(color="black", width=2),
    )

    fig.update_xaxes(
        ticktext=[ "O1s", "O2s", "O2p", "O2p", "O2p", "H1s", "H1s",],
        tickvals=np.arange(11),
    )
    fig.update_yaxes(
        ticktext=["O1s", "O2s", "O2p", "O2p", "O2p", "H1s", "H1s"],
        tickvals=np.arange(11),
    )

    return fig

In [17]:
# Visualise computed Hamiltonian
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_waters_dataset

hamiltonians = []
angles = []
structures = []
overlap_matrices = []
coefficients = []
eigenvalues = []

for doc in collection.find({}).limit(10):
    hamiltonians.append(doc["fock_matrices"][0])
    overlap_matrices.append(doc["overlap_matrices"][0])
    structure = Molecule.from_dict(doc["structures"][0])
    structures.append(structure)
    coefficients.append(doc["coeff_matrices"][0])
    angles.append(doc["angles"])
    eigenvalues.append(doc["eigenvalues"][0])

hamiltonians = np.array(hamiltonians)
overlap_matrices = np.array(overlap_matrices)
coefficients = np.array(coefficients)
angles = np.array(angles)
eigenvalues = np.array(eigenvalues)
# Unsqueeze to add a batch dimension
eigenvalues = eigenvalues[..., np.newaxis, :]

fig = px.imshow(
    hamiltonians[:, 1, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='DFT computed Hamiltonian for a rotated water molecule', title_x=0.5)
fig.show()


In [18]:

fig = px.imshow(
    overlap_matrices[:, 1, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='DFT computed overlap matrix for a rotated water molecule', title_x=0.5)
fig.show()


In [20]:
fig = px.imshow(
    eigenvalues[:,0,...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
)
fig.update_layout(title_text='DFT computed eigenvalues for a rotated water molecule', title_x=0.5)
fig.update_yaxes(tickmode='array', tickvals=np.arange(11))
fig.update_xaxes(tickmode='array', tickvals=np.arange(11))
fig.show()

In [59]:
fig = px.imshow(
    np.abs(coefficients[:, 1, ...]), animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='DFT computed coefficient matrix for a rotated water molecule', title_x=0.5)
# Remove tick labels from the yaxis
# Change y-label to be `eigenvalues index`
fig.update_xaxes(title_text="Eigenvalues index")
# Change the x-axis labels to be the integral labels
fig.update_xaxes(
    ticktext=np.arange(8),
    tickvals=np.arange(8),
)

fig.show()


In order to be useful to understand chemical reactions, we sub-diagonalise the Hamiltonian, which allows us to look at inter-atomic orbital level interactions.

In [34]:
indices_all = [[0, 1, 2, 3, 4], [5, ], [ 6]]

hamiltonians_sub = []
overlap_matrices_sub = []
coupling = []

for i in range(hamiltonians.shape[0]):
    spin_up_H = hamiltonians[i, 0, ...]
    spin_up_S = overlap_matrices[i, 0, ...]
    H_r = spin_up_H
    S_r = spin_up_S
    for indices in indices_all:
        H_r, S_r, eigenval = subdiagonalize_matrix(indices, H_r, S_r) 
    hamiltonians_sub.append(H_r)
    overlap_matrices_sub.append(S_r)

    # Subtract the diagonal elements from the Hamiltonian
    H_r = H_r - np.diag(np.diag(H_r))
    coupling.append(H_r)

# Plot the subdiagonalized Hamiltonian
hamiltonians_sub = np.array(hamiltonians_sub)
overlap_matrices_sub = np.array(overlap_matrices_sub)
coupling = np.array(coupling)

fig = px.imshow(
    hamiltonians_sub[:, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"), 
    range_color=[-0.5, 0.5],
)
add_bounds_to_figure(fig)

fig.update_layout(title_text='Subdiagonalised Hamiltonian (H)', title_x=0.5)

# Draw an axvline and an axhline on the first row and column
# fig.show()

In [35]:
# Plot the overlap matrix
fig = px.imshow(
    overlap_matrices_sub[:, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='Subdiagonalised Overlap Matrix (S)', title_x=0.5)
fig.show()


In order to determine _how_ the Hamiltonian changes as a function of rotation $\left (\alpha, \beta, \gamma \right)$, we create the Wigner-D matrix.

$$
\boldsymbol{H}\left( \alpha, \beta, \gamma \right) = D\left(\alpha, \beta, \gamma\right) \boldsymbol{H} D\left(\alpha, \beta, \gamma\right)^T
$$

In [36]:
irreps_hamiltonian = o3.Irreps("1x0e+1x0e+1x1o+1x0e+1x0e")

D_matrices = []
for idx, angle in enumerate(angles):
    alpha, beta, gamma = angle

    rotation_matrix = rotate_three_dimensions(alpha, beta, gamma)
    rotation_matrix = torch.tensor(rotation_matrix)
    if idx == 0:
        rotation_matrix_0 = rotation_matrix

    # Reference the rotation matrix to the first one
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T

    D_matrix = irreps_hamiltonian.D_from_matrix(rotation_matrix)
    D_matrices.append(D_matrix)

D_matrices = torch.stack(D_matrices)
# Convert to numpy array
D_matrices = D_matrices.detach().numpy()

fig = px.imshow(
    D_matrices, animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)

fig.update_layout(title_text='Wigner Matrix (D)', title_x=0.5)
add_bounds_to_figure(fig)
fig.show()

In [37]:
# Referencing all the angles to the first angle, compute the Hamiltonian in the rotated basis
hamiltonians_rotated = []
for i in range(len(angles)):
    hamiltonian_rotated = D_matrices[i] @ hamiltonians[0, ..., :, :] @ D_matrices[i].T
    hamiltonians_rotated.append(hamiltonian_rotated)

hamiltonians_rotated = np.array(hamiltonians_rotated)
hamiltonians_rotated_diff = hamiltonians_rotated - hamiltonians

fig = px.imshow(
    hamiltonians_rotated_diff[:, 0, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)

add_bounds_to_figure(fig)

fig.update_layout(title_text='Difference between computed and rotated H', title_x=0.5)

fig.show()

In [44]:
# Check the rotation of the overlap matrix
overlap_matrices_rotated = []
for i in range(len(angles)):
    overlap_matrix_rotated = D_matrices[i] @ overlap_matrices[0, ..., :, :] @ D_matrices[i].T
    overlap_matrices_rotated.append(overlap_matrix_rotated)

overlap_matrices_rotated = np.array(overlap_matrices_rotated)
overlap_matrices_rotated_diff = overlap_matrices_rotated - overlap_matrices

fig = px.imshow(
    overlap_matrices_rotated_diff[:, 0, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
add_bounds_to_figure(fig)
fig.show()

In [79]:
# Check the rotation of the coefficient matrix
coefficients_rotated = []
coefficient_rotated_diff = []
for i in range(len(angles)):
    # take the transpose of the coefficient matrix  for the last two dimensions
    coefficient_rotated =  coefficients[0, 0, ..., :].T @ D_matrices[i]
    coefficients_rotated.append(coefficient_rotated)
    _coefficients_rotated_diff = coefficient_rotated - coefficients[i, 0, :, :].T
    coefficient_rotated_diff.append(_coefficients_rotated_diff) 

coefficients_rotated = np.array(coefficients_rotated)
coefficient_rotated_diff = np.array(coefficient_rotated_diff)

fig = px.imshow(
    coefficient_rotated_diff[:, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Eigenvalue Index", color="Value"),
    range_color=[-1, 1],
)
fig.show()
